<h1>Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods

In [1]:
import pandas as pd

In [2]:
path = 'companies.csv'
df = pd.read_csv(path)

In [3]:
df.head()

,Rank,Name,Industry,Revenue (USD millions),Profit (USD millions),Employees,Country,Ref
0,1,Walmart,Retail,"$514,405","Increase $6,670","2,200,000",United States United States,[5]
1,2,Sinopec Group,Oil and gas,"$414,649","Increase $5,845","619,151",China China,[6]
2,3,Royal Dutch Shell,Oil and gas,"$396,556","Increase $23,352","81,000",Netherlands Netherlands / United Kingdom Unite...,[7]
3,4,China National Petroleum,Oil and gas,"$392,976","Increase $2,270","1,382,401",China China,[8]
4,5,State Grid,Electricity,"$387,056","Increase $8,174","917,717",China China,[9]


<h2>Data

<h3>Data cleaning

In [4]:
#delete and rename columns
del df['Ref']
df.rename(columns = {'Revenue (USD millions)':'Revenue_USDM', 'Profit (USD millions)':'Profit_USDM'}, inplace=True)
df.head(10)

,Rank,Name,Industry,Revenue_USDM,Profit_USDM,Employees,Country
0,1,Walmart,Retail,"$514,405","Increase $6,670","2,200,000",United States United States
1,2,Sinopec Group,Oil and gas,"$414,649","Increase $5,845","619,151",China China
2,3,Royal Dutch Shell,Oil and gas,"$396,556","Increase $23,352","81,000",Netherlands Netherlands / United Kingdom Unite...
3,4,China National Petroleum,Oil and gas,"$392,976","Increase $2,270","1,382,401",China China
4,5,State Grid,Electricity,"$387,056","Increase $8,174","917,717",China China
5,6,Saudi Aramco,Oil and gas,"$355,905","Increase $110,974","76,418",Saudi Arabia Saudi Arabia
6,7,BP,Oil and gas,"$303,738","Increase $9,383","73,000",United Kingdom United Kingdom
7,8,ExxonMobil,Oil and gas,"$290,212","Increase $20,840","71,000",United States United States
8,9,Volkswagen,Automotive,"$278,341","Increase $14,332","664,496",Germany Germany
9,10,Toyota,Automotive,"$272,612","Increase $16,982","370,870",Japan Japan


In [5]:
#remove duplicates countrie's names values in the column Country
duplicates_val = list(df.Country)
from collections import Counter 
non_dupl_val = []
def remov_duplicates(input): 
  
    # split input string separated by space 
    input = input.split(" ") 
  
    # joins two adjacent elements in iterable way 
    for i in range(0, len(input)): 
        input[i] = "".join(input[i]) 
  
    # now create dictionary using counter method 
    # which will have strings as key and their  
    # frequencies as value 
    UniqW = Counter(input) 
  
    # joins two adjacent elements in iterable way 
    s = " ".join(UniqW.keys()) 
    non_dupl_val.append(s)
for i in duplicates_val:
    remov_duplicates(i)

df["Country"] = non_dupl_val
df.head()

,Rank,Name,Industry,Revenue_USDM,Profit_USDM,Employees,Country
0,1,Walmart,Retail,"$514,405","Increase $6,670","2,200,000",United States
1,2,Sinopec Group,Oil and gas,"$414,649","Increase $5,845","619,151",China
2,3,Royal Dutch Shell,Oil and gas,"$396,556","Increase $23,352","81,000",Netherlands / United Kingdom
3,4,China National Petroleum,Oil and gas,"$392,976","Increase $2,270","1,382,401",China
4,5,State Grid,Electricity,"$387,056","Increase $8,174","917,717",China


we have the value "Netherlands / United Kingdom" separated by "/", let's duplicate the row for each county

In [6]:
df['Country'] = df['Country'].astype("str")
df = (df.set_index(['Rank', 'Name', 'Industry', 'Revenue_USDM', 'Profit_USDM', 'Employees'])['Country']
       .str.split('/', expand=True)
       .stack()
       .reset_index(name='Country'))
df.head()

,Rank,Name,Industry,Revenue_USDM,Profit_USDM,Employees,level_6,Country
0,1,Walmart,Retail,"$514,405","Increase $6,670","2,200,000",0,United States
1,2,Sinopec Group,Oil and gas,"$414,649","Increase $5,845","619,151",0,China
2,3,Royal Dutch Shell,Oil and gas,"$396,556","Increase $23,352","81,000",0,Netherlands
3,3,Royal Dutch Shell,Oil and gas,"$396,556","Increase $23,352","81,000",1,United Kingdom
4,4,China National Petroleum,Oil and gas,"$392,976","Increase $2,270","1,382,401",0,China


In [7]:
#we don't need the column "level_6"
del df['level_6']
df.head()

,Rank,Name,Industry,Revenue_USDM,Profit_USDM,Employees,Country
0,1,Walmart,Retail,"$514,405","Increase $6,670","2,200,000",United States
1,2,Sinopec Group,Oil and gas,"$414,649","Increase $5,845","619,151",China
2,3,Royal Dutch Shell,Oil and gas,"$396,556","Increase $23,352","81,000",Netherlands
3,3,Royal Dutch Shell,Oil and gas,"$396,556","Increase $23,352","81,000",United Kingdom
4,4,China National Petroleum,Oil and gas,"$392,976","Increase $2,270","1,382,401",China


Later we will cluster the companies based on its revenue, so we need to check the column type

In [8]:
df.dtypes

Rank             int64
Name            object
Industry        object
Revenue_USDM    object
Profit_USDM     object
Employees       object
Country         object
dtype: object

the revenue column type is "object", we need to convert it to int.<br>
first we need to eliminate the $ sign, then the comma sign "," so we can convert the amounts to int type.

In [9]:
df['Revenue_USDM'] = df['Revenue_USDM'].str.slice_replace(0, 1, '')
df['Revenue_USDM'] = df['Revenue_USDM'].str.slice_replace(3, 4, '')
df.head()

,Rank,Name,Industry,Revenue_USDM,Profit_USDM,Employees,Country
0,1,Walmart,Retail,514405,"Increase $6,670","2,200,000",United States
1,2,Sinopec Group,Oil and gas,414649,"Increase $5,845","619,151",China
2,3,Royal Dutch Shell,Oil and gas,396556,"Increase $23,352","81,000",Netherlands
3,3,Royal Dutch Shell,Oil and gas,396556,"Increase $23,352","81,000",United Kingdom
4,4,China National Petroleum,Oil and gas,392976,"Increase $2,270","1,382,401",China


In [10]:
df['Revenue_USDM']=df['Revenue_USDM'].astype("int")
df.head()

,Rank,Name,Industry,Revenue_USDM,Profit_USDM,Employees,Country
0,1,Walmart,Retail,514405,"Increase $6,670","2,200,000",United States
1,2,Sinopec Group,Oil and gas,414649,"Increase $5,845","619,151",China
2,3,Royal Dutch Shell,Oil and gas,396556,"Increase $23,352","81,000",Netherlands
3,3,Royal Dutch Shell,Oil and gas,396556,"Increase $23,352","81,000",United Kingdom
4,4,China National Petroleum,Oil and gas,392976,"Increase $2,270","1,382,401",China


Now the revenue column type is an "int"

In [11]:
df.dtypes

Rank             int64
Name            object
Industry        object
Revenue_USDM     int32
Profit_USDM     object
Employees       object
Country         object
dtype: object

<h3>Data completing

We will add the columns longitude and latitude to our dataframe

first of all, lets get the longitude and the latitude of the countries in our dataframe

In [12]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

In [13]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'DF5BMQDPRRUJZCCFY13LCPEA432ASRS43MSIQCIH3HYOCXV2' # your Foursquare ID
CLIENT_SECRET = 'TN2SPLTNFABGGC2IUAFYDLKLROFWFPS2XEEGWACU12JKC1QU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
list_longitude=[]
list_latitude=[]
#to define the longitude and latitude from the adress 
for i in list(df.Country):
    address = i #we are in this adress
    geolocator = Nominatim(user_agent="foursquare_agent") #We will name our agent foursquare_agent
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print("Latitude {}, Longitude {}...".format(latitude,longitude))
    list_latitude.append(latitude)
    list_longitude.append(longitude)
df["Latitude"]=list_latitude
df["Longitude"]=list_longitude
df.head()

Latitude 39.7837304, Longitude -100.4458825...
Latitude 35.000074, Longitude 104.999927...
Latitude 52.5001698, Longitude 5.7480821...
Latitude 54.7023545, Longitude -3.2765753...
Latitude 35.000074, Longitude 104.999927...
Latitude 35.000074, Longitude 104.999927...
Latitude 25.6242618, Longitude 42.3528328...
Latitude 54.7023545, Longitude -3.2765753...
Latitude 39.7837304, Longitude -100.4458825...
Latitude 51.0834196, Longitude 10.4234469...
Latitude 36.5748441, Longitude 139.2394179...
Latitude 39.7837304, Longitude -100.4458825...
Latitude 39.7837304, Longitude -100.4458825...
Latitude 39.7837304, Longitude -100.4458825...
Latitude 39.7837304, Longitude -100.4458825...
Latitude 36.638392, Longitude 127.6961188...
Latitude 46.7985624, Longitude 8.2319736...
Latitude 39.7837304, Longitude -100.4458825...
Latitude 51.0834196, Longitude 10.4234469...
Latitude 39.7837304, Longitude -100.4458825...
Latitude 46.603354, Longitude 1.8883335...
Latitude 35.000074, Longitude 104.999927...
L

,Rank,Name,Industry,Revenue_USDM,Profit_USDM,Employees,Country,Latitude,Longitude
0,1,Walmart,Retail,514405,"Increase $6,670","2,200,000",United States,39.783730,-100.445882
1,2,Sinopec Group,Oil and gas,414649,"Increase $5,845","619,151",China,35.000074,104.999927
2,3,Royal Dutch Shell,Oil and gas,396556,"Increase $23,352","81,000",Netherlands,52.500170,5.748082
3,3,Royal Dutch Shell,Oil and gas,396556,"Increase $23,352","81,000",United Kingdom,54.702354,-3.276575
4,4,China National Petroleum,Oil and gas,392976,"Increase $2,270","1,382,401",China,35.000074,104.999927


<h3>Data visualization

In this section, we will create a world map with the localization of each company 

In [14]:
map_world = folium.Map(location=[latitude, longitude], zoom_start=1)
for lat, lng, country, name in zip(df['Latitude'], df['Longitude'], df['Country'], df['Name']): # add markers to map
    label = '{}, {}'.format(name, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_world)  
map_world

<h2>Cluster countries based on the revenues of its companies

To know the counties having the largest companies based on the revenue, we have to cluster them.
we chose to cluster them in five clusters.

let's group the companies by countries

In [15]:
df_grp = df.groupby(['Country', 'Latitude', 'Longitude'])['Revenue_USDM'].sum().reset_index()
df_grp.sort_values('Revenue_USDM', ascending=False,inplace =True)
df_grp.reset_index(inplace=True)
df_grp.drop([5,8],0,inplace=True)
df_grp[""]=1
df_grp

,index,Country,Latitude,Longitude,Revenue_USDM,
0,14,United States,39.783730,-100.445882,4169049,1
1,1,China,35.000074,104.999927,2263520,1
2,3,Germany,51.083420,10.423447,602635,1
3,4,Japan,36.574844,139.239418,561157,1
4,0,United Kingdom,54.702354,-3.276575,396556,1
6,8,Saudi Arabia,25.624262,42.352833,355905,1
7,2,France,46.603354,1.888334,309684,1
9,7,Russia,64.686314,97.745306,250447,1
10,10,South Korea,36.638392,127.696119,221579,1
11,11,Switzerland,46.798562,8.231974,219754,1


to fit the k-means alogorithm we have to prepare the array x containg the sum of the revenues from the companies for each country.

In [16]:
x=df_grp.values[:,4:5]
x

array([[4169049],
       [2263520],
       [602635],
       [561157],
       [396556],
       [355905],
       [309684],
       [250447],
       [221579],
       [219754],
       [180744],
       [175617],
       [175009]], dtype=object)

Now lets cluster the countries based on the revenues from the companies to each country.<br>
we chose to cluster the countries to 5 clusters.

In [17]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(x)
# dataframe that includes the cluster as well as the top 10 venues for each neighborhood
df_grp.insert(0, 'Cluster Labels', kmeans.labels_)


Now lets visualise the the result of the clustering alogorithm "kmeans" n the datarame df_grp.

In [18]:
df_grp

,Cluster Labels,index,Country,Latitude,Longitude,Revenue_USDM,
0,1,14,United States,39.783730,-100.445882,4169049,1
1,2,1,China,35.000074,104.999927,2263520,1
2,3,3,Germany,51.083420,10.423447,602635,1
3,3,4,Japan,36.574844,139.239418,561157,1
4,4,0,United Kingdom,54.702354,-3.276575,396556,1
6,4,8,Saudi Arabia,25.624262,42.352833,355905,1
7,4,2,France,46.603354,1.888334,309684,1
9,0,7,Russia,64.686314,97.745306,250447,1
10,0,10,South Korea,36.638392,127.696119,221579,1
11,0,11,Switzerland,46.798562,8.231974,219754,1


for a better exploring, here is the groups of clusters.<br>
Please check the report for further analysis.

In [19]:
df_clust = df_grp.groupby(['Cluster Labels','Country']).sum()
df_clust

index   Latitude   Longitude  Revenue_USDM   
Cluster Labels Country                                                       
0              Netherlands          5  52.500170    5.748082        175009  1
               Russia               7  64.686314   97.745306        250447  1
               Singapore            9   1.357107  103.819499        180744  1
               South Korea         10  36.638392  127.696119        221579  1
               Switzerland         11  46.798562    8.231974        219754  1
               Taiwan              12  23.598298  120.835363        175617  1
1              United States       14  39.783730 -100.445882       4169049  1
2              China                1  35.000074  104.999927       2263520  1
3              Germany              3  51.083420   10.423447        602635  1
               Japan                4  36.574844  139.239418        561157  1
4               United Kingdom      0  54.702354   -3.276575        396556  1
               France               2  46.603354    1.888334        309684  1
               Saudi Arabia         8  25.624262   42.352833        355905  1

<h2>Clustering Visualization with folium

Finally, let's visualize the resulting clusters

In [20]:
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors

In [21]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=1)
# set color scheme for the clusters  
x = np.arange(kclusters)  
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_grp['Latitude'], df_grp['Longitude'], df_grp['Country'], df_grp['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters
